In [0]:
### CHH Yang 2018, ICML CompbioWorkshop 2018
### contact: huckiyang @ github
## https://github.com/huckiyang/EvoluGeneNet-Adjacency-Matrix-Visualizer
#
# If you find this useful in your work, please consider citing the following reference:
# @article{yang2018learning,
#   title={Learning Functions in Large Networks requires Modularity and produces Multi-Agent Dynamics},
#   author={Yang, CH and Ooi, Rise and Hiscock, Tom and Eguiluz, Victor and Tegner, Jesper},
#   journal={arXiv preprint arXiv:1807.03001},
#   year={2018}
# }



import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

#########################ICML-IJCAI, Huck Yang, Rise Ooi, et al. 2018###########################
## GeneNet Hyperparameters
B       = 16       #batch size
N       = 300       #timesteps
S       = 3         #number of species
dt      = 0.01      #time interval
L1      = 0.003      #L1 regularization parameter

## Genetic Algorithm Hyperparameters
POP_SIZE = 12               # population size
CLONES_NUM = 2               # Num of times of cloning, note: assert(POP_SIZE // CLONES_NO == 0)
CROSS_RATE = 0.8              # mating probability (DNA crossover) (1 here because there are duplicates anyway)
CROSS_POINT_RATE = 0.5     # DNA points crossover rate
REVERSE_CROSS_RATE = 0.5     # Reverse crossover printing rate
STEP_SIZE = 2               # mutation intensity
N_GENERATIONS = 100          # generation number
RAND_ITERATIONS = 100      # Initialization iterations
GA_ITERATIONS = 100         # GA iterations
PRUNE_ITERATIONS = 10     # Prune iterations
IWV = 5                  # Initial value of the initial_weights_data elements
#LOW_STEP_BOUND = 1 - STEP_SIZE
#HIGH_STEP_BOUND = 1 + STEP_SIZE

# Parameters (Derived from the hyperparameters, do not change anything)
DNA_SIZE = S*S              # DNA length
MUTATION_RATE = 0.5/DNA_SIZE        # mutation probability
if(POP_SIZE%CLONES_NUM == 0):
    SELECT_NUM = POP_SIZE//CLONES_NUM
else:
    raise AssertionError()

####################################################################
##Data 
input_      = tf.placeholder(tf.float32, shape=[N, B, S])
output_     = tf.placeholder(tf.float32, shape=[B])
initial_    = tf.placeholder(tf.float32, shape=[B, S])

####################################################################
##Variables to optimize
W = tf.Variable(tf.random_normal([S,S], mean = 0, stddev = 0.1, dtype = tf.float32))
A = tf.Variable(1.0, dtype = tf.float32)

####################################################################
##Nonlinearity chosen
def phi(x):
    return 1/(tf.exp(-x)+1)

####################################################################
##ODE function
def simulate(input_, initial_, W):
    output = tf.scan(lambda o,i: o + dt*(phi(tf.matmul(o,W))-o+i),
                            elems = input_,
                            initializer = initial_,
                            swap_memory = True)
    return output


output = simulate(input_, initial_, W)

####################################################################
## Training function
relevantOutput  = A*output[N-1,:,1]
fullOutput 		= A*output[N-1,:,:]

## Without regularization
cost = tf.reduce_mean(tf.square(((relevantOutput - output_)))) 
train_step = tf.train.AdamOptimizer(learning_rate=0.2, beta1=0.98, beta2=0.999, epsilon=1e-08).minimize(cost)
## With regularization
costL1 = tf.reduce_mean(tf.square(((relevantOutput - output_)))) + L1*tf.reduce_mean(tf.abs(W)) 
train_stepL1 = tf.train.AdamOptimizer(learning_rate=0.2, beta1=0.98, beta2=0.999, epsilon=1e-08).minimize(costL1)

####################################################################
## Initial Weights Data
initial_weights_data = []
# # 0: BI type French Flag
# initial_weights_data.append([[0, 0, 0],
#                             [0, IWV, -IWV],
#                             [-IWV, IWV, 0]])
# # 1: OD type French Flag
# initial_weights_data.append([[0, 0, -IWV],
#                             [IWV, 0, 0],
#                             [-IWV, IWV, 0]])
# # 2: IFF type French Flag
# initial_weights_data.append([[0, 0, 0],
#                             [IWV, 0, 0],
#                             [IWV, -IWV, IWV]])
# # 3: MI type French Flag
# initial_weights_data.append([[0, 0, 0],
#                             [IWV, 0, -IWV],
#                             [IWV, -IWV, 0]])
# # 4: FO type French Flag
# initial_weights_data.append([[IWV, 0, 0],
#                             [-IWV, 0, IWV],
#                             [-IWV, -IWV, IWV]])
# # 5: Classical French Flag
# initial_weights_data.append([[0, 0, 0],
#                             [-IWV, IWV, 0],
#                             [-IWV, -IWV, IWV]])

## Direct Method
# 0: BI type French Flag
initial_weights_data.append([[-4.62, -7.50, -7.77],
                            [-9.59, 16.07, -15.26],
                            [-18.12, 5.88, -5.99]])
# 1: OD type French Flag
initial_weights_data.append([[-43.88, -6.45, -16.36],
                            [46.57, 14.83, -11.99],
                            [-35.90, 1.80, -14.28]])
# 2: IFF type French Flag
initial_weights_data.append([[0.17, 4.85, -1.03],
                            [0.11, 0.13, -2.23],
                            [-0.06, -7.48, 4.28]])
# 3: MI type French Flag
initial_weights_data.append([[-1.28, -2.27, -5.70],
                            [-0.15, 8.92, 0.29],
                            [0.02, -5.45, 8.46]])
# 4: FO type French Flag
initial_weights_data.append([[-1.30, -2.19, -5.44],
                            [-0.59, 8.55, 0.52],
                            [-0.13, -5.41, 7.79]])
# 5: Classical French Flag
initial_weights_data.append([[-2.82079792, 0.7832284, -5.42571878],
                            [-0.22299732, -0.26842004, 0.46841779],
                            [-3.14926434, -5.17239904, 8.41496181]])



In [0]:


####################################################################
## Training data - here for a "stripe"
def newBatch(plot=False):
    initialVal      = 0.1*np.ones([B,S])
    if(plot):
        inputNoNoise = np.linspace(0,2,B)
    else:
        inputNoNoise    = np.random.uniform(0,2,B)
    inputVal        = inputNoNoise.reshape(B,1) * np.random.normal(loc=1.0, scale = 0.0001, size=[N,B,S])
    inputVal[:,:,1:S] = 0.0
    outputNoNoise   = np.zeros_like(inputNoNoise)
    #outputNoNoise[inputNoNoise > 1.0] = 1.0 # Original PlotAll
    outputNoNoise[np.logical_and(inputNoNoise > 0.5,inputNoNoise < 1.5 )] = 1.0
    return [inputVal, outputNoNoise, initialVal]



In [0]:
####################################################################
## Training model function
# Original GeneNet trainModel with tweaks
def trainModel(iterations,regularize=False,prune=False,pruneLimit=1,printing=False):
    mask = np.abs(sess.run(W)) > pruneLimit
    for i in range(iterations):
        [inputVal, outputVal, initialVal] = newBatch()
        if(regularize):
            sess.run(train_stepL1, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
        else:
            sess.run(train_step, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
        if(printing):
            ww = sess.run(cost, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
            weights = sess.run(W)
            print([ww, i/iterations],end="\r")
        if(prune):
            applyMask = W.assign(W*mask)
            sess.run(applyMask)
            
    ww = sess.run(cost, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
    outputW = sess.run(W)
    outputA = sess.run(A)
    return ww, outputW, outputA

In [0]:


# Original GeneNet randomized trainModel outputting weights and cost
def trainRandomModel(iterations,initial=-1,regularize=False,prune=False,pruneLimit=1,printing=False):
    # There is no failing mechanism if initial is less than -1 or more than 5
    if(initial <= -1):
        randomize_W = W.assign(tf.random_normal([S,S], mean = 0, stddev = 0.1, dtype = tf.float32))
        sess.run(randomize_W)
    elif(initial>=0 and initial<=5):
        set_W = W.assign(initial_weights_data[initial])
        sess.run(set_W)
    else:
        raise AssertionError()
        
    randomize_A = A.assign(1.0)
    sess.run(randomize_A)
    
    mask = np.abs(sess.run(W)) > pruneLimit
    for i in range(iterations):
        [inputVal, outputVal, initialVal] = newBatch()
        if(regularize):
            sess.run(train_stepL1, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
        else:
            sess.run(train_step, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
        if(printing):
            ww = sess.run(cost, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
            outputW = sess.run(W)
            print([ww, i/iterations],end="\r")
        if(prune):
            applyMask = W.assign(W*mask)
            sess.run(applyMask)
    
    ww = sess.run(cost, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
    outputW = sess.run(W)
    outputA = sess.run(A)
    return ww, outputW, outputA



In [0]:
# Incomplete function. # TODO: Uses previously trained weights and cost to retrain them. 
def trainEvolvingModel(iterations, evolved_w, evolved_a, regularize=False,prune=False,pruneLimit=1,printing=False):
    set_W = W.assign(evolved_w)
    set_A = A.assign(evolved_a)
    sess.run(set_W)
    sess.run(set_A)
    mask = np.abs(sess.run(W)) > pruneLimit
    for i in range(iterations):
        [inputVal, outputVal, initialVal] = newBatch()
        if(regularize):
            sess.run(train_stepL1, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
        else:
            sess.run(train_step, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
        if(printing):
            ww = sess.run(cost, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
            outputW = sess.run(W)
            print([ww, i/iterations],end="\r")
        if(prune):
            applyMask = W.assign(W*mask)
            sess.run(applyMask)
    
    ww = sess.run(cost, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
    outputW = sess.run(W)
    outputA = sess.run(A)
    return ww, outputW, outputA

In [0]:


####################################################################
## Simulate model
def simulateModel():
    [inputVal, outputVal, initialVal] = newBatch(plot=True)
    finalOutput = sess.run(fullOutput, feed_dict = {input_: inputVal, initial_: initialVal, output_: outputVal})
    print(sess.run(W))
    print(sess.run(A))
    for j in np.arange(0,S):
        plt.plot(finalOutput[:,j])
    plt.show()



In [0]:
####################################################################
## GA Functions
# TODO: Upgrade the selection method to use rank method with diversity selector built in
def selection(w_storage, a_storage):    # selecting most fit half of population (no diversity built in)
    # Assuming children number is even
    # Assuming w_storage are sorted
    w_ret = []
    a_ret = []
    for i in range(SELECT_NUM):
        for _ in range(CLONES_NUM):
            w_ret.append(w_storage[i])
            a_ret.append(a_storage[i])
    return w_ret, a_ret

def crossover(population):     # mating process (genes crossover)
    for i in range(POP_SIZE):
        if np.random.rand() < CROSS_RATE:
        #if True: # Since there are duplicates, there's 20% chance it won't cross over anyway.
            crossover_target = np.random.randint(POP_SIZE)
            for j in range(DNA_SIZE):
                if np.random.rand() < CROSS_POINT_RATE:
                    if np.random.rand() < REVERSE_CROSS_RATE:
                        # printing from crossover_target WITH reverse printing (exchange)
                        reverse_tmp = population[i][j]
                        population[i][j] = population[crossover_target][j]
                        population[crossover_target][j] = reverse_tmp
                    else:
                        # no reverse printing
                        population[i][j] = population[crossover_target][j]
                    
            
    return population

#TODO: Should we mutate A?
def mutate(population):
    for i in range(POP_SIZE):
        for j in range(DNA_SIZE):
            if np.random.rand() < MUTATION_RATE:
                # If STEP_SIZE is 1, it will only scales from 0~1
                # Default STEP_SIZE is 2, but should be increased according to type of problem
                # (-1)**np.random.randint(2) gives either -1 or 1
                population[i][j] *= STEP_SIZE*np.random.rand() * (-1)**np.random.randint(2)
                # New mutation way:
                #population[i][j] *= STEP_SIZE * -1

    return population

In [0]:
# Roll & Unrolling
def unroll(_list):
    ret = []
    for i in range(S):
        for j in range(S):
            ret.append(_list[i][j])
            
    return ret

def roll(_list):
    ret = []
    _list.append(0) # delete if possible
    for i in range(S):
        ret.append(_list[i*S:i*S+3])

    return ret


In [0]:


#Triple mergesort sorting by alist
#TODO: convert cost_storage, w_storage, and a_storage into one single dictionary instead of three lists
def triple_mergesort(alist, blist, clist):
    #print("Splitting ",alist)
    if len(alist)>1:
        mid = len(alist)//2
        lefthalf_a = alist[:mid]
        lefthalf_b = blist[:mid]
        lefthalf_c = clist[:mid]
        righthalf_a = alist[mid:]
        righthalf_b = blist[mid:]
        righthalf_c = clist[mid:]
         

        triple_mergesort(lefthalf_a, lefthalf_b, lefthalf_b)
        triple_mergesort(righthalf_a, righthalf_b, righthalf_c)

        i=0
        j=0
        k=0
        while i < len(lefthalf_a) and j < len(righthalf_a):
            if lefthalf_a[i] < righthalf_a[j]:
                alist[k]=lefthalf_a[i]
                blist[k]=lefthalf_b[i]
                clist[k]=lefthalf_c[i]
                i=i+1
            else:
                alist[k]=righthalf_a[j]
                blist[k]=righthalf_b[j]
                clist[k]=righthalf_c[j]
                j=j+1
            k=k+1

        while i < len(lefthalf_a):
            alist[k]=lefthalf_a[i]
            blist[k]=lefthalf_b[i]
            clist[k]=lefthalf_c[i]
            i=i+1
            k=k+1

        while j < len(righthalf_a):
            alist[k]=righthalf_a[j]
            blist[k]=righthalf_b[j]
            clist[k]=righthalf_c[j]
            j=j+1
            k=k+1



In [0]:


####################################################################
## Determine type
# type_table = np.zeros(3**9)
# def update_type_table(trained_weights, type_table):
#     flat_w = []
#     for i in range(S):
#         for j in range(S):
#             flat_w.append(trained_weights[i][j])
#     for i in range(POP_SIZE):
#         index = 0
#         for j in range(DNA_SIZE):
#             if (flat_w[j] <= -1):
#                 index += 0
#             elif (flat_w[j] >= 1):
#                 index += 3**j
#             else:
#                 index += 2*(3**j)
#         # note: index's binaries are in reverse
#         type_table[index] += 1
        
#     return type_table;

## Determine not classical type straight away and stores the child number and generation
type_dict_table = {}
def determine_type(tw, type_dict_table):
    # tw means trained_weights
    return type_dict_table



In [0]:


####################################################################
## GA-Layered Train model
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

##Global Storage
# Note that the weights are not converted to anything else from real form.
cost_storage = []     # Cost lookup table
w_storage = [];       # Stores children weights
a_storage = [];       # Stores A


# Initialize random children and stores them to weights_storage
# Assume POP_SIZE is 12
for p in range(POP_SIZE):
    t1 = time.time()
    print("\nRandomly initialized Child #",p)
    child_cost, child_w, child_a = trainRandomModel(iterations=RAND_ITERATIONS, initial=p//2)
    #simulateModel()
    #print("After regularize...")
    #child_cost, child_w, child_a = trainModel(iterations=PRUNE_ITERATIONS, regularize=True)
    #simulateModel()
    #print("After pruning...")
    #child_cost, child_w, child_a = trainModel(iterations=PRUNE_ITERATIONS, prune=True)
    simulateModel()
    print("Cost:", child_cost)
    cost_storage.append(child_cost)
    w_storage.append(child_w)
    a_storage.append(child_a)
    t2 = time.time()
    print("Time: ", t2-t1)

#print(cost_storage)
#print(w_storage)
#print(a_storage)

